In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10
from google.colab import drive
drive.mount('/content/drive')


np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.9 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



170498071/170498071 [==============================] - 13s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
teacher_model_path = '/content/base/LeNetCIFAR10_final.tf'
techer_model = load_model(teacher_model_path)

In [ ]:
model_name = 'LeNet'
dataset = 'CIFAR10'

# Student 0.1 Alpha

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.3896 - accuracy: 0.5114
Test Loss: 1.3896385431289673, Test Accuracy: 0.5113999843597412
Epoch 2:
313/313 [==============================] - 2s 5ms/step - loss: 1.2514 - accuracy: 0.5614
Test Loss: 1.2513569593429565, Test Accuracy: 0.5613999962806702
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1559 - accuracy: 0.5949
Test Loss: 1.1559396982192993, Test Accuracy: 0.5949000120162964
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0866 - accuracy: 0.6203
Test Loss: 1.0865685939788818, Test Accuracy: 0.6202999949455261
Epoch 5:
313/313 [==============================] - 2s 5ms/step - loss: 1.0353 - accuracy: 0.6403
Test Loss: 1.0352933406829834, Test Accuracy: 0.6402999758720398
Epoch 6:
313/313 [==============================] - 1s 5ms/step - loss: 1.0036 - accuracy: 0.6536
Test Loss: 1.0036042928695679, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same.zip'

# student 0.1 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)


313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.3927 - accuracy: 0.4969
Test Loss: 1.3927242755889893, Test Accuracy: 0.4968999922275543
Epoch 2:
313/313 [==============================] - 2s 5ms/step - loss: 1.2546 - accuracy: 0.5484
Test Loss: 1.2546470165252686, Test Accuracy: 0.5483999848365784
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1701 - accuracy: 0.5835
Test Loss: 1.1700600385665894, Test Accuracy: 0.5835000276565552
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.1085 - accuracy: 0.6097
Test Loss: 1.1085050106048584, Test Accuracy: 0.6097000241279602
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0621 - accuracy: 0.6254
Test Loss: 1.0621140003204346, Test Accuracy: 0.6254000067710876
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 1.0160 - accuracy: 0.6453
Test Loss: 1.0160430669784546, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_2.zip'

# Student 0.1 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)


313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.4243 - accuracy: 0.4932
Test Loss: 1.4243232011795044, Test Accuracy: 0.49320000410079956
Epoch 2:
313/313 [==============================] - 1s 4ms/step - loss: 1.2695 - accuracy: 0.5516
Test Loss: 1.2695013284683228, Test Accuracy: 0.5515999794006348
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1953 - accuracy: 0.5823
Test Loss: 1.1952534914016724, Test Accuracy: 0.5823000073432922
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.1434 - accuracy: 0.6031
Test Loss: 1.1434063911437988, Test Accuracy: 0.6031000018119812
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0884 - accuracy: 0.6263
Test Loss: 1.0884301662445068, Test Accuracy: 0.6262999773025513
Epoch 6:
313/313 [==============================] - 2s 5ms/step - loss: 1.0499 - accuracy: 0.6393
Test Loss: 1.0498945713043213, 

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_3", 'zip', "student_same_3")
colab_link = "/content/student_same_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_3.zip'

# Student same Alpha 0.5

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 2s 4ms/step - loss: 1.3898 - accuracy: 0.5118
Test Loss: 1.3897937536239624, Test Accuracy: 0.5117999911308289
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2506 - accuracy: 0.5596
Test Loss: 1.2505754232406616, Test Accuracy: 0.5595999956130981
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1508 - accuracy: 0.5981
Test Loss: 1.1508499383926392, Test Accuracy: 0.5981000065803528
Epoch 4:
313/313 [==============================] - 1s 5ms/step - loss: 1.0858 - accuracy: 0.6227
Test Loss: 1.0858230590820312, Test Accuracy: 0.6226999759674072
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0382 - accuracy: 0.6393
Test Loss: 1.0382475852966309, Test Accuracy: 0.6392999887466431
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 1.0046 - accuracy: 0.6494
Test Loss: 1.0046297311782837, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_50.zip'

# Student 0.5 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 2s 5ms/step - loss: 1.3790 - accuracy: 0.5040
Test Loss: 1.37897789478302, Test Accuracy: 0.5040000081062317
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2476 - accuracy: 0.5536
Test Loss: 1.2475544214248657, Test Accuracy: 0.553600013256073
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 1.1626 - accuracy: 0.5863
Test Loss: 1.1626299619674683, Test Accuracy: 0.5863000154495239
Epoch 4:
313/313 [==============================] - 2s 5ms/step - loss: 1.0930 - accuracy: 0.6129
Test Loss: 1.092957615852356, Test Accuracy: 0.6129000186920166
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 1.0431 - accuracy: 0.6356
Test Loss: 1.043083667755127, Test Accuracy: 0.6355999708175659
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.9993 - accuracy: 0.6501
Test Loss: 0.9993215799331665, Test A

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

# Student 0.5 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 4ms/step - loss: 1.4125 - accuracy: 0.4939
Test Loss: 1.4125194549560547, Test Accuracy: 0.49390000104904175
Epoch 2:
313/313 [==============================] - 1s 4ms/step - loss: 1.2653 - accuracy: 0.5552
Test Loss: 1.2652581930160522, Test Accuracy: 0.5551999807357788
Epoch 3:
313/313 [==============================] - 2s 5ms/step - loss: 1.1939 - accuracy: 0.5822
Test Loss: 1.193942666053772, Test Accuracy: 0.5821999907493591
Epoch 4:
313/313 [==============================] - 2s 5ms/step - loss: 1.1340 - accuracy: 0.6041
Test Loss: 1.133960247039795, Test Accuracy: 0.6040999889373779
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0893 - accuracy: 0.6258
Test Loss: 1.0893019437789917, Test Accuracy: 0.6258000135421753
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 1.0508 - accuracy: 0.6368
Test Loss: 1.0508496761322021, Te

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_50_3.zip'

# Student same Alpha 0.9

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.3859 - accuracy: 0.5111
Test Loss: 1.385861873626709, Test Accuracy: 0.5110999941825867
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2458 - accuracy: 0.5624
Test Loss: 1.2457538843154907, Test Accuracy: 0.5623999834060669
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1457 - accuracy: 0.5959
Test Loss: 1.1457195281982422, Test Accuracy: 0.5958999991416931
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0801 - accuracy: 0.6217
Test Loss: 1.0800893306732178, Test Accuracy: 0.6216999888420105
Epoch 5:
313/313 [==============================] - 2s 5ms/step - loss: 1.0345 - accuracy: 0.6386
Test Loss: 1.0344932079315186, Test Accuracy: 0.6385999917984009
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 1.0035 - accuracy: 0.6529
Test Loss: 1.003484845161438, Tes

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_90.zip'

# student 0.9 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)

model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

mkdir: cannot create directory ‘/content/student_same_90_2/’: File exists
313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.3747 - accuracy: 0.5050
Test Loss: 1.3747040033340454, Test Accuracy: 0.5049999952316284
Epoch 2:
313/313 [==============================] - 1s 4ms/step - loss: 1.2423 - accuracy: 0.5529
Test Loss: 1.2423115968704224, Test Accuracy: 0.5529000163078308
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1585 - accuracy: 0.5859
Test Loss: 1.1585019826889038, Test Accuracy: 0.5859000086784363
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0979 - accuracy: 0.6119
Test Loss: 1.0978584289550781, Test Accuracy: 0.6118999719619751
Epoch 5:
313/313 [==============================] - 2s 8ms/step - loss: 1.0458 - accuracy: 0.6337
Test Loss: 1.045790672302246, Test Accuracy: 0.6337000131607056
Epoch 6:
313/313 [==============================] - 1s 3m

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_90_2.zip'

# student 0.9 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)

model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

mkdir: cannot create directory ‘/content/student_same_90_3/’: File exists
313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 1.4229 - accuracy: 0.4917
Test Loss: 1.4228923320770264, Test Accuracy: 0.4916999936103821
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2644 - accuracy: 0.5548
Test Loss: 1.2643755674362183, Test Accuracy: 0.5547999739646912
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1932 - accuracy: 0.5844
Test Loss: 1.1931699514389038, Test Accuracy: 0.5843999981880188
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.1393 - accuracy: 0.6029
Test Loss: 1.1392600536346436, Test Accuracy: 0.6029000282287598
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0942 - accuracy: 0.6219
Test Loss: 1.0941767692565918, Test Accuracy: 0.6219000220298767
Epoch 6:
313/313 [==============================] - 2s 5

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C10/student_same_90_3.zip'

---